<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter

Импортируем библиотеки

In [1]:
import pandas as pd # для вывода результата запроса
import psycopg2 # для подключения к PostgreSQL
import urllib.parse as req # для открытия url-адреса

# Для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

Создаём соединение с заданными параметрами:

In [3]:
connection = psycopg2.connect(
    dbname = DBNAME,
    user = USER,
    host = HOST,
    password = PASSWORD,
    port = PORT
)

## 1. Предварительный анализ данных

### 1.1 Подсчитаем количество вакансий в базе. 

In [4]:
# Текст запроса
query_1_1 = f'''SELECT
                COUNT(*)
            FROM public.vacancies
         '''

In [5]:
# Результат запроса
result_1_1 = pd.read_sql_query(query_1_1, connection)
result_1_1

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/1865541962.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_1_1 = pd.read_sql_query(query_1_1, connection)


,count
0,49197


### 1.2 Подсчитаем количество работодателей.

In [6]:
# Текст запроса
query_1_2 = f'''SELECT 
                COUNT(*)
            FROM public.employers
         '''

In [7]:
# Результат запроса
result_1_2 = pd.read_sql_query(query_1_2, connection)
result_1_2

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/18672583.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_1_2 = pd.read_sql_query(query_1_2, connection)


,count
0,23501


### 1.3 Подсчитаем количество регионов.

In [8]:
# Текст запроса
query_1_3 = f'''SELECT 
                       COUNT(*)
                   FROM public.areas
                '''

In [9]:
# Результат запроса
result_1_3 = pd.read_sql_query(query_1_3, connection)
result_1_3

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/42247351.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_1_3 = pd.read_sql_query(query_1_3, connection)


,count
0,1362


### 1.4 Подсчитаем количество сфер деятельности.

In [10]:
# Текст запроса
query_1_4 = f'''SELECT 
                            COUNT(*)
                        FROM public.industries
                     '''

In [11]:
# Результат запроса
result_1_4 = pd.read_sql_query(query_1_4, connection)
result_1_4

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/1449171635.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_1_4 = pd.read_sql_query(query_1_4, connection)


,count
0,294


**Вывод по детальному анализу вакансий:** По результату запроса мы видим, что в базе данных представлено 49197 вакансии, которые включают в себя 294 сферы деятельности в 1362 регионов, а количество компаний-работодателей равно 23501.

## 2. Детальный анализ вакансий

### 2.1 Подсчитаем количество вакансий в каждом регионе. Выберем пятёрку лидеров по количеству вакансий.

In [20]:
# Текст запроса
query_2_1 = f'''SELECT 
                          a.name "Название города",
                          COUNT(DISTINCT v.name) "Количество вакансий"
                      FROM 
                         public.areas a 
                      JOIN public.vacancies v ON a.id = v.area_id
                      GROUP BY a.name
                      ORDER BY COUNT(DISTINCT v.name) DESC
                      LIMIT 5
                   '''

In [21]:
# Результат запроса
result_2_1 = pd.read_sql_query(query_2_1, connection)
result_2_1

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/1829692516.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Название города,Количество вакансий
0,Москва,3894
1,Санкт-Петербург,2143
2,Новосибирск,1501
3,Минск,1497
4,Алматы,1421


In [29]:
# Строим столбчатую диаграмму
fig = px.bar(
    data_frame=result_2_1,
    x='Название города',
    y='Количество вакансий',
    orientation='v',
    color='Название города',
    height=500,
    width=1000,
    title='Топ 5 городов по количеству вакансий'
)

# Отображаем его
fig.show()

### 2.2 Подсчитаем количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой.

In [30]:
# Текст запроса
query_2_2 = f'''SELECT 
                          COUNT(*)
                      FROM public.vacancies
                      WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
                   '''

In [31]:
# Результат запроса
result_2_2 = pd.read_sql_query(query_2_2, connection)
result_2_2

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/2340204899.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,24073


### 2.3 Подсчитаем средние значения для нижней и верхней границы зарплатной вилки.

In [32]:
# Текст запроса
query_2_3 = f'''SELECT  
                            ROUND(AVG(salary_to)) "Верхняя граница зарплатной вилки",
                            ROUND(AVG(salary_from)) "Нижняя граница зарплатной вилки"
                     FROM public.vacancies
                  '''

In [33]:
# Результат запроса
result_2_3 = pd.read_sql_query(query_2_3, connection)
result_2_3

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/2640410187.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Верхняя граница зарплатной вилки,Нижняя граница зарплатной вилки
0,110537.0,71065.0


### 2.4 Подсчитаем количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Выведем пару, которая находится на втором месте по популярности.

In [45]:
# Текст запроса
query_2_4 = f'''SELECT 
                          COUNT(DISTINCT name) "Количество вакансий",
                          schedule "Тип графика",
                          employment "Тип трудоустройства"
                      FROM public.vacancies
                      GROUP BY 2, 3
                      ORDER BY 1 DESC
                      OFFSET 1 LIMIT 1
                   '''           

In [46]:
# Результат запроса
result_2_4 = pd.read_sql_query(query_2_4, connection)
result_2_4

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/1338231588.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Количество вакансий,Тип графика,Тип трудоустройства
0,3574,Удаленная работа,Полная занятость


### 2.5 Выведем значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [39]:
# Текст запроса
query_2_5 = f'''SELECT 
                               COUNT(DISTINCT name) "Количество вакансий",
                               experience "Требуемый опыт работы" 
                           FROM public.vacancies
                           GROUP BY 2
                           ORDER BY 1 ASC
                        '''

In [40]:
# Результат запроса
result_2_5 = pd.read_sql_query(query_2_5, connection)
result_2_5

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/963506210.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Количество вакансий,Требуемый опыт работы
0,871,Более 6 лет
1,3243,Нет опыта
2,7151,От 3 до 6 лет
3,11963,От 1 года до 3 лет


In [41]:
# Строим столбчатую диаграмму
fig = px.bar(
    data_frame=result_2_5,
    x='Требуемый опыт работы',
    y='Количество вакансий',
    orientation='v',
    color='Требуемый опыт работы',
    height=500,
    width=1000,
    title='Зависимость требуемого опыта работы от количества вакансий'
)

# Отображаем его
fig.show()

**Вывод по детальному анализу вакансий:** По результатам запроса можно заключить, что лидером по количеству вакансий среди всех городов является Москва (количество вакансий: 3894). Это обусловлено тем, что данный регион является самым большим по количеству населения, а так же самым развитым. Касаемо заработных плат, то значение верхней границы зароплатной вилки составляет 110537 рублей, что на 39472 руб выше нижнего значения (71065 рублей). Также мы видим, что среди соискателей наиболее востребованными являются специалисты со стажем от 1 до 3-х лет, данный показатель можно объяснить тем, что в эту категорию, по большей части, попадают недавно выпустившиеся из учебных заведений специалисты с уже имеющимся стажем.

## 3. Анализ работодателей

### 3.1 Выведем работодателей, которые находятся на первом и пятом месте по количеству вакансий.

In [42]:
# Текст запроса
query_3_1 = f'''(SELECT 
                e.name "Название работодателя",
                COUNT(*) "Количество вакансий" 
            FROM public.employers e
            JOIN public.vacancies v ON e.id = v.employer_id
            GROUP BY e.id
            ORDER BY 2 DESC
            LIMIT 1)    
            
            UNION ALL
            
            (SELECT
                e.name "Название работодателя",
                COUNT(*) "Количество вакансий"
            FROM public.employers e
            JOIN public.vacancies v ON e.id = v.employer_id
            GROUP BY e.id
            ORDER BY 2 DESC
            OFFSET 4 LIMIT 1)
         '''

In [43]:
# Результат запроса
result_3_1 = pd.read_sql_query(query_3_1, connection)
result_3_1

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_5044/2655606297.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Название работодателя,Количество вакансий
0,Яндекс,1933
1,Газпром нефть,331


### 3.2 Для каждого региона выведем количество работодателей и вакансий в нём. Среди регионов, в которых нет вакансий, найдём тот, в котором наибольшее количество работодателей.

In [4]:
# Текст запроса
query_3_2 = f'''SELECT 
                a.name "Название региона",
                COUNT(DISTINCT e.id) "Количество работодателей",
                COUNT(DISTINCT v.id) "Количество вакансий"
            FROM 
                public.areas a
            LEFT JOIN public.employers e ON a.id = e.area
            LEFT JOIN public.vacancies v ON a.id = v.area_id
            WHERE v.id IS NULL
            GROUP BY a.id
            ORDER BY 2 DESC
            LIMIT 1
         '''

In [5]:
# Результат запроса
result_3_2 = pd.read_sql_query(query_3_2, connection)
result_3_2

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/3994287390.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_3_2 = pd.read_sql_query(query_3_2, connection)


,Название региона,Количество работодателей,Количество вакансий
0,Россия,410,0


### 3.3 Для каждого работодателя посчитаем количество регионов, в которых он публикует свои вакансии. Выберем максимальное значение из получившегося списка.

In [6]:
# Текст запроса
query_3_3 = f'''SELECT 
                e.name "Название компании",
                COUNT(DISTINCT v.area_id) "Количество регионов"
            FROM 
                public.employers e
            JOIN public.vacancies v ON e.id = v.employer_id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 1  
         '''

In [7]:
# Результат запроса
result_3_3 = pd.read_sql_query(query_3_3, connection)
result_3_3

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/1442407713.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_3_3 = pd.read_sql_query(query_3_3, connection)


,Название компании,Количество регионов
0,Яндекс,181


### 3.4 Подсчитаем количество работодателей, у которых не указана сфера деятельности.

In [8]:
# Текст запроса
query_3_4 = f'''SELECT 
                COUNT(DISTINCT e.id)
            FROM public.employers e
            LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
            WHERE ei.industry_id IS NULL
         '''

In [9]:
# Результат запроса
result_3_4 = pd.read_sql_query(query_3_4, connection)
result_3_4

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/4184335811.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_3_4 = pd.read_sql_query(query_3_4, connection)


,count
0,8419


### 3.5 Выведем название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [14]:
# Текст запроса
query_3_5 = f'''SELECT 
                e.name "Название компании",
                COUNT(ei.industry_id) "Количество сфер деятельности"
            FROM
                public.employers e
            JOIN public.employers_industries ei ON e.id = ei.employer_id
            GROUP BY 1
            HAVING COUNT(ei.industry_id) = 4
            ORDER BY 1
            OFFSET 2 LIMIT 1
         '''

In [15]:
# Результат запроса
result_3_5 = pd.read_sql_query(query_3_5, connection)
result_3_5

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/1537891369.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_3_5 = pd.read_sql_query(query_3_5, connection)


,Название компании,Количество сфер деятельности
0,2ГИС,4


### 3.6 Вычислим количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения».

In [16]:
# Текст запроса
query_3_6 = f'''SELECT 
                DISTINCT i.name "Название сферы деятельности",
                COUNT(ei.employer_id) "Количество работодателей"
            FROM 
                public.industries i 
            JOIN public.employers_industries ei ON i.id = ei.industry_id
            WHERE i.name = 'Разработка программного обеспечения' 
            GROUP BY 1
         '''

In [17]:
# Результат запроса
result_3_6 = pd.read_sql_query(query_3_6, connection)
result_3_6

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/1946524360.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_3_6 = pd.read_sql_query(query_3_6, connection)


,Название сферы деятельности,Количество работодателей
0,Разработка программного обеспечения,3553


### 3.7 Для компании «Яндекс» выведем список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавим строку Total с общим количеством вакансий компании, собранных в этой таблице.

In [20]:
# Прописываем путь 
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)

tables = pd.read_html(url) # читаем таблицу
cities_df = tables[1] # создаём DataFrame
cities_list = list(cities_df['Город']) # создаём список городов-миллионников
cities_list[13] = 'Воронеж'

In [21]:
# Текст запроса
query_3_7 = f'''SELECT
                    a.name "Название региона",
                    COUNT(v.id) "Количество вакансий"
                FROM public.vacancies v 
                JOIN public.areas a ON a.id = v.area_id 
                JOIN public.employers e ON v.employer_id = e.id
                WHERE e.name = 'Яндекс' AND a.name in {tuple(cities_list)}
                GROUP BY 1
                
                UNION ALL
                
                SELECT
                    'Total',
                    COUNT(*)
                FROM public.vacancies v 
                JOIN public.areas a ON a.id = v.area_id 
                JOIN public.employers e ON v.employer_id = e.id
                WHERE e.name = 'Яндекс' AND a.name in {tuple(cities_list)}
                
                ORDER BY 2
             '''

In [23]:
# Результат запроса
result_3_7 = pd.read_sql_query(query_3_7, connection)
result_3_7

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/923661798.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_3_7 = pd.read_sql_query(query_3_7, connection)


,Название региона,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


In [28]:
# Для лучшего отображения количества вакансий на диаграмме создадим датафрейм без строки "Total", и передадим его в код графика
vacancies_by_cities = result_3_7.drop(labels=[16], axis=0)

# Строим столбчатую диаграмму
fig = px.bar(
    data_frame=vacancies_by_cities,
    x='Название региона',
    y='Количество вакансий',
    orientation='v',
    color='Название региона',
    height=500,
    width=1000,
    title='Количество вакансий компании "Яндекс" по регионам'
)

# Отображаем его
fig.show()

**Вывод по анализу работодателей:** По полученным данным мы видим, что среди всех работодателей (23501 компания) на первом месте по числу вакансий находится компания *"Яндекс"* с количеством вакансий 1933, эта же компания охватывает наибольшее число регионов. Такой результат можно объяснить как масштабами самой компании, так и количеством вакансий и позиций на различные должности. Что касается до самих регионов, в которых *"Яндекс"* размещает объявления, то среди представленных городов, наибольшее число вакансий компании в количестве 54-х размещено в Москве. Данный показатель можно объяснить в основном количеством населения данного региона (по количеству населения Москва занимает первое место), а следовательно, количество соискателей тут будет так же наибольшее. 

## 4. Предметный анализ

### 4.1 Вычислим количество вакансий, которые имеют отношения к данным.

In [29]:
# Текст запроса
query_4_1 = f'''SELECT
                    COUNT(*)
                FROM public.vacancies
                WHERE LOWER(name) LIKE '%data%' OR LOWER(name) LIKE '%данн%'
             '''

In [30]:
# Результат запроса
result_4_1 = pd.read_sql_query(query_4_1, connection)
result_4_1

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/1728018148.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


### 4.2 Вычислим количество вакансий для начинающего дата-сайентиста

In [31]:
# Текст запроса
query_4_2 = f'''SELECT 
                COUNT(*)
            FROM public.vacancies
            WHERE ((name ILIKE '%data scientist%')
                OR (name ILIKE '%data science%')
                OR (name ILIKE '%исследователь данных%')
                OR (name ILIKE '%machine learning%')
                OR (name ILIKE '%машинн%обучен%%')
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                )
                
                AND
                
                  ((name ILIKE '%junior%')
                OR (experience = 'Нет опыта')
                OR (employment = 'Стажировка'))
         '''

In [32]:
# Результат запроса
result_4_2 = pd.read_sql_query(query_4_2, connection)
result_4_2

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/4254207785.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


### 4.3 Вычислим количество вакансий для *DS*, в которых в качестве ключевого навыка указан *SQL* или *postgres*

In [33]:
# Текст запроса
query_4_3 = f'''SELECT 
                COUNT(*)
            FROM public.vacancies
            WHERE ((name ILIKE '%data scientist%')
                OR (name ILIKE '%data science%')
                OR (name ILIKE '%исследователь данных%')
                OR (name ILIKE '%machine learning%')
                OR (name ILIKE '%машинн%обучен%%')
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                )
                
            AND
                
                (key_skills LIKE '%SQL%' OR key_skills LIKE '%postgres%')
         '''

In [34]:
# Результат запроса
result_4_3 = pd.read_sql_query(query_4_3, connection)
result_4_3

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/2319930842.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,201


### 4.4 Вычислим количество вакансий, в которых в качестве ключевого навыка указан *Python*

In [35]:
# Текст запроса
query_4_4 = f'''SELECT 
                COUNT(*)
            FROM public.vacancies
            WHERE ((name ILIKE '%data scientist%')
                OR (name ILIKE '%data science%')
                OR (name ILIKE '%исследователь данных%')
                OR (name ILIKE '%machine learning%')
                OR (name ILIKE '%машинн%обучен%%')
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                )
                
            AND
                
                (key_skills LIKE '%Python%')
         '''

In [36]:
# Результат запроса
result_4_4 = pd.read_sql_query(query_4_4, connection)
result_4_4

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/741748853.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,351


### 4.5 Вычислим количество ключевых навыков, которые среднем указывают в вакансиях для *DS*. Ответ округлим до двух знаков после точки-разделителя.

In [37]:
# Текст запроса
query_4_5 = f'''SELECT 
                ROUND(AVG(LENGTH(key_skills) - LENGTH(replace(key_skills, CHR(9), '')) + 1 ), 2)
            FROM public.vacancies
            WHERE ((name ILIKE '%data scientist%')
                OR (name ILIKE '%data science%')
                OR (name ILIKE '%исследователь данных%')
                OR (name ILIKE '%machine learning%')
                OR (name ILIKE '%машинн%обучен%%')
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                )
                
                AND NOT key_skills IS NULL
         '''

In [38]:
# Результат запроса
result_4_5 = pd.read_sql_query(query_4_5, connection)
result_4_5

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/2417089674.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round
0,6.41


### 4.6 Вычислим зарплату для *DS*, которую в среднем указывают для каждого типа требуемого опыта.

In [41]:
# Текст запроса
query_4_6 = f'''SELECT 
                    experience "Требуемый опыт",
                    ROUND((AVG(coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))) / 2) "Средняя зарплата"
                FROM public.vacancies
                WHERE ((name ILIKE '%data scientist%')
                OR (name ILIKE '%data science%')
                OR (name ILIKE '%исследователь данных%')
                OR (name ILIKE '%machine learning%')
                OR (name ILIKE '%машинн%обучен%%')
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                )
                
                GROUP BY 1
             '''

In [42]:
# Результат запроса
result_4_6 = pd.read_sql_query(query_4_6, connection)
result_4_6

C:\Users\ilyaz\AppData\Local\Temp/ipykernel_4432/2435532526.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Требуемый опыт,Средняя зарплата
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


**Вывод по предметному анализу:** Из результата запроса мы видим, что среди 49197 вакансий, представленных в базе, к данным имеют отношения только 1771 вакансия, среди которых 51 относится непосредственно к начинающему дата-сайентисту. Среди ключевых навыков прописанных в более чем 200 вакансиях, которыми должен обладать DS выделяются владение SQL или postgres а так же обязательное владение языком Python. Что до заработных плат, то средняя зп начинающего DS равна 74643 рубля, по достижении опыта от 1 года: 139675 рублей а специалисты со стажем от 3 до 6 лет могут зарабатывать от 243115 рублей в месяц. Такие высокие доходы можно объяснить высокой востребованностью квалифицированных специалистов по данным, которая с каждым годом становится только выше и будет расти в ближайшей перспективе.

## Общий вывод по проекту

В ходе работы над проектом были проведены следующие исследования базы данных: вычислили общее количество вакансий, их количество по компаниям, регионам; исследовали работодателей, их количество, распределение по количеству вакансий; а так же отдельно провели исследование вакансий по профессии Data Scientist. В дальнейшем можно провести углубленное исследование наиболее востребованных вакансий, их количество по регионам, вилку зп.

## 5. Финал

In [4]:
# Закрываем соединение
connection.close()